# Interpretable_MLmodel_urban_flood_susceptibility

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from scipy.sparse.linalg import cg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import os
import sys
import glob
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.tree 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import math
import torch
# import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import lime.lime_tabular
import shap
# from interpret_community.tabular_explainer import TabularExplainer
from captum.attr import IntegratedGradients
# import tensorflow as tf
# from tensorflow.keras import backend as K
import time
import lime.lime_tabular
import shap
# from interpret_community.tabular_explainer import TabularExplainer
from captum.attr import IntegratedGradients
# import tensorflow as tf
# from tensorflow.keras import backend as K
import time
from sklearn.preprocessing import MinMaxScaler



In [ ]:
import random
m=32
random.seed(m)
np.random.seed(m)
torch.manual_seed(m)

In [ ]:
# import data
train_data_original=pd.read_csv('train_data.csv')
test_data_original=pd.read_csv('test_data.csv')

In [ ]:
# standardize data

scaler = MinMaxScaler()
train_data = train_data_original.copy()  
train_data.iloc[:, 1:9] = scaler.fit_transform(train_data_original.iloc[:, 1:9])  

test_data = test_data_original.copy() 
test_data.iloc[:, 1:9] = scaler.fit_transform(test_data_original.iloc[:, 1:9]) 

train_data


In [ ]:
# concatenate train and test data
data_train_test=pd.concat([train_data,test_data])
df=data_train_test.iloc[:,1:-1]
df

In [ ]:
# all features

all_features = df.iloc[:, :-1]

all_features

In [ ]:
# train data 
X_train = train_data.iloc[:,1:-1]

X_train

In [ ]:
Y_train = train_data['Suscept_3']

Y_train

In [ ]:
X_test = test_data.iloc[:,1:-2]

X_test

In [ ]:
Y_test = test_data['Suscept_3']
Y_test

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

print(pd.value_counts(Y_train))

print(pd.value_counts(Y_test))

In [ ]:
def print_test_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

def print_train_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_train) == Y_train)/len(Y_train)))
    time.sleep(0.5) # to let the print get out before any progress bars
shap.initjs()

In [ ]:
all_features.describe()

## Model_training

MLP

In [ ]:
'''
# nn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc


mlp = MLPClassifier()


param_grid = {
    
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    #'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['relu', 'tanh'],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [100, 200, 500],
    #'alpha': [0.0001, 0.05],
    
}


cv = StratifiedKFold(n_splits=5, shuffle=True,random_state=42
)


grid_search_mlp = GridSearchCV(mlp, param_grid=param_grid,
                           cv=cv, scoring='recall')


grid_search_mlp.fit(X_train, Y_train)


print('Best parameters:', grid_search_mlp.best_params_)
print('Best AUC:', grid_search_mlp.best_score_)


print("Best parameters: ", grid_search_mlp.best_params_)
print("Best score: ", grid_search_mlp.best_score_)



params_mlp = grid_search_mlp.best_params_
best_params_mlp = MLPClassifier(**params_mlp)
best_params_mlp.fit(X_train, Y_train)
'''

In [ ]:

# nn

model_mlp_new = MLPClassifier( activation= 'tanh', hidden_layer_sizes= (100,), learning_rate= 'constant', max_iter= 500, solver= 'adam')

# train model on training set
model_mlp_new.fit(X_train, Y_train)



LR

In [ ]:
'''
# LR
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd

lr = LogisticRegression()
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 500],
}

grid_search_lr = GridSearchCV(lr, param_grid, scoring='recall', cv=cv)
grid_search_lr.fit(X_train, Y_train)

print("Best parameters: ", grid_search_lr.best_params_)
print("Best score: ", grid_search_lr.best_score_)

params_lr = grid_search_lr.best_params_
best_params_lr =  LogisticRegression(**params_lr)
best_params_lr.fit(X_train, Y_train)

'''

In [ ]:

# lr

model_lr_new = LogisticRegression(C= 10, max_iter= 100, penalty= 'l1', solver= 'liblinear')

model_lr_new.fit(X_train, Y_train)


RF

In [ ]:
'''

rf = RandomForestClassifier()


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search_rf = GridSearchCV(rf, param_grid, scoring='recall', cv=cv)

grid_search_rf.fit(X_train, Y_train)

print("Best parameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)

params_rf = grid_search_rf.best_params_
best_params_rf =  RandomForestClassifier(**params_rf)
best_params_rf.fit(X_train, Y_train)
'''

In [ ]:

# rf
model_rf_new = RandomForestClassifier(max_depth= 20, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 200)

# train model on training set
model_rf_new.fit(X_train, Y_train)


XGBoost

In [ ]:
'''

xgb_model = xgb.XGBClassifier()
    

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1],
}


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


grid_search_xgb = GridSearchCV(xgb_model,
                           param_grid=param_grid,
                           scoring='recall',
                           cv=cv)

grid_search_xgb.fit(X_train, Y_train)


print("Best parameters: ", grid_search_xgb.best_params_)
print("Best score: ", grid_search_xgb.best_score_)


params_xgb =grid_search_xgb.best_params_
best_params_xgb = xgb.XGBClassifier(**params_xgb)
best_params_xgb.fit(X_train, Y_train)
'''

In [ ]:
# xgb
model_xgb_new = xgb.XGBClassifier(colsample_bytree=1, gamma= 0.2, learning_rate= 0.1, max_depth= 7, n_estimators= 200, subsample= 0.9)

# train model on training set
model_xgb_new.fit(X_train, Y_train)

## 3. Model_preformance

In [ ]:


import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc



model_lr = model_lr_new
model_xgb =  model_xgb_new
model_mlp =  model_mlp_new
model_rf =  model_rf_new

'''

model_lr = best_params_lr
model_xgb =  best_params_xgb
model_mlp =  best_params_mlp
model_rf =  best_params_rf
'''


Y_test_pred1 = model_lr.predict(X_test)
Y_test_pred2 = model_xgb.predict(X_test)
Y_test_pred3 = model_mlp.predict(X_test)
Y_test_pred4 = model_rf.predict(X_test)


accuracy_train1 = accuracy_score(Y_train, model_lr.predict(X_train))
accuracy_train2 = accuracy_score(Y_train, model_xgb.predict(X_train))
accuracy_train3 = accuracy_score(Y_train, model_mlp.predict(X_train))
accuracy_train4 = accuracy_score(Y_train, model_rf.predict(X_train))

accuracy_test1 = accuracy_score(Y_test, model_lr.predict(X_test))
accuracy_test2 = accuracy_score(Y_test, model_xgb.predict(X_test))
accuracy_test3 = accuracy_score(Y_test, model_mlp.predict(X_test))
accuracy_test4 = accuracy_score(Y_test, model_rf.predict(X_test))

precision_train1 = precision_score(Y_train, model_lr.predict(X_train))
precision_train2 = precision_score(Y_train, model_xgb.predict(X_train))
precision_train3 = precision_score(Y_train, model_mlp.predict(X_train))
precision_train4 = precision_score(Y_train, model_rf.predict(X_train))

precision_test1 = precision_score(Y_test, Y_test_pred1)
precision_test2 = precision_score(Y_test, Y_test_pred2)
precision_test3 = precision_score(Y_test, Y_test_pred3)
precision_test4 = precision_score(Y_test, Y_test_pred4)

recall_train1 = recall_score(Y_train, model_lr.predict(X_train))
recall_train2 = recall_score(Y_train, model_xgb.predict(X_train))
recall_train3 = recall_score(Y_train, model_mlp.predict(X_train))
recall_train4 = recall_score(Y_train, model_rf.predict(X_train))

recall_test1 = recall_score(Y_test, Y_test_pred1)
recall_test2 = recall_score(Y_test, Y_test_pred2)
recall_test3 = recall_score(Y_test, Y_test_pred3)
recall_test4 = recall_score(Y_test, Y_test_pred4)

f1score_train1 = f1_score(Y_train, model_lr.predict(X_train))
f1score_train2 = f1_score(Y_train, model_xgb.predict(X_train))
f1score_train3 = f1_score(Y_train, model_mlp.predict(X_train))
f1score_train4 = f1_score(Y_train, model_rf.predict(X_train))

f1score_test1 = f1_score(Y_test, Y_test_pred1)
f1score_test2 = f1_score(Y_test, Y_test_pred2)
f1score_test3 = f1_score(Y_test, Y_test_pred3)
f1score_test4 = f1_score(Y_test, Y_test_pred4)


results_train = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (train)': [accuracy_train1, accuracy_train2, accuracy_train3],
    'Precision (train)': [precision_train1, precision_train2, precision_train3],
    'Recall (train)': [recall_train1, recall_train2, recall_train3],
    'F1 score (train)': [f1score_train1, f1score_train2, f1score_train3],
}
results_test = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (test)': [accuracy_test1, accuracy_test2, accuracy_test3],
    'Precision (test)': [precision_test1, precision_test2, precision_test3],
    'Recall (test)': [recall_test1, recall_test2, recall_test3],
    'F1 score (test)': [f1score_test1, f1score_test2, f1score_test3],
}


df_results_train = pd.DataFrame(results_train)
df_results_test = pd.DataFrame(results_test)


In [ ]:

results_train_rf = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp','Model_rf'],
    'Accuracy (train)': [accuracy_train1, accuracy_train2, accuracy_train3, accuracy_train4],
    'Precision (train)': [precision_train1, precision_train2, precision_train3,precision_train4],
    'Recall (train)': [recall_train1, recall_train2, recall_train3,recall_train4],
    'F1 score (train)': [f1score_train1, f1score_train2, f1score_train3,f1score_train4],
}
results_test_rf = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp','Model_rf'],
    'Accuracy (test)': [accuracy_test1, accuracy_test2, accuracy_test3,accuracy_test4],
    'Precision (test)': [precision_test1, precision_test2, precision_test3, precision_test4],
    'Recall (test)': [recall_test1, recall_test2, recall_test3,recall_test4],
    'F1 score (test)': [f1score_test1, f1score_test2, f1score_test3,f1score_test4],
}


df_results_train_rf = pd.DataFrame(results_train_rf)
df_results_test_rf = pd.DataFrame(results_test_rf)

In [ ]:
df_results_train_rf

In [ ]:
df_results_test_rf

In [ ]:
# Compute performance metrics
Y_test1 = Y_test.ravel() 
xgb_loss = sklearn.metrics.log_loss(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_loss = sklearn.metrics.log_loss(Y_test1, model_lr.predict_proba(X_test)[:,1])
nn_loss = sklearn.metrics.log_loss(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_loss = sklearn.metrics.log_loss(Y_test1, model_rf.predict_proba(X_test)[:,1])

print("XGBoost_loss", xgb_loss)
print("LR_loss", linear_loss)
print("MLP_loss", nn_loss)
print("RF_loss", nn_loss)
print()

xgb_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_roc_auc = sklearn.metrics.roc_auc_score(Y_test1, model_rf.predict_proba(X_test)[:,1])

print("XGBoost_roc_auc", xgb_roc_auc)
print("LR_roc_auc", linear_roc_auc)
print("MLP_roc_auc", dnn_roc_auc)
print("RF_roc_auc", rf_roc_auc)

print()

xgb_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_pr_auc = sklearn.metrics.average_precision_score(Y_test1, model_rf.predict_proba(X_test)[:,1])
print("XGBoost_pr_auc", xgb_pr_auc)
print("LR_pr_auc", linear_pr_auc)
print("MLP_pr_auc", dnn_pr_auc)
print("RF_pr_auc", rf_pr_auc)
print()

xgb_fpr, xgb_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_xgb.predict_proba(X_test)[:,1])
linear_fpr, linear_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_lr.predict_proba(X_test)[:,1])
dnn_fpr, dnn_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_mlp.predict_proba(X_test)[:,1])
rf_fpr, rf_tpr, thresholds = sklearn.metrics.roc_curve(Y_test1, model_rf.predict_proba(X_test)[:,1])



pl.plot(rf_fpr, rf_tpr, label="AUC RF %.3f" % rf_roc_auc)
pl.plot(dnn_fpr, dnn_tpr, label="AUC MLP %.3f" % dnn_roc_auc)
pl.plot(linear_fpr, linear_tpr, label="AUC LR %.3f" % linear_roc_auc)
pl.legend()
pl.title("ROC curves on test data")
pl.show()

xgb_prec, xgb_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_xgb.predict_proba(X_test)[:,1])
dnn_prec, dnn_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_mlp.predict_proba(X_test)[:,1])
linear_prec, linear_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_lr.predict_proba(X_test)[:,1])
rf_prec, rf_recall, thresholds = sklearn.metrics.precision_recall_curve(Y_test1, model_rf.predict_proba(X_test)[:,1])


pl.plot(dnn_recall, dnn_prec, label="MLP %.3f" % dnn_pr_auc)
pl.plot(linear_recall, linear_prec, label="LR %.3f" % linear_pr_auc)
pl.plot(rf_recall, rf_prec, label="RF %.3f" % rf_pr_auc)

pl.legend()
pl.title("Precision-Recall curves on test data")
pl.show()

In [ ]:
# save output
output_test_lr=pd.concat([test_data,pd.Series(model_lr.predict_proba(X_test)[:,1],name='probability_lr')], axis=1)
output_test_xgb=pd.concat([test_data,pd.Series(model_xgb.predict_proba(X_test)[:,1],name='probability_xgb')], axis=1)
output_test_mlp=pd.concat([test_data,pd.Series(model_mlp.predict_proba(X_test)[:,1],name='probability_mlp')], axis=1)
output_test_rf=pd.concat([test_data,pd.Series(model_rf.predict_proba(X_test)[:,1],name='probability_rf')], axis=1)



output_test_lr.to_csv('output_test_lr.csv', index=False) 
output_test_xgb.to_csv('output_test_xgb.csv', index=False) 
output_test_mlp.to_csv('output_test_mlp.csv', index=False) 
output_test_rf.to_csv('output_test_rf.csv', index=False) 



### LIME and SHAP feature contribution 

In [ ]:
X_test = X_test.reset_index(drop=True)
X_test.iloc[0]

In [ ]:
# SHAP-RF

explainer = shap.TreeExplainer(model_rf_new)


shap_values = explainer.shap_values(X_test)

feature_important_rf = pd.DataFrame(shap_values[1], columns=X_test.columns)

feature_important_rf.to_csv('feature_important_shap_rf.csv', index=False)

feature_important_rf

In [ ]:
# LIME-RF

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train), 
    feature_names=X_train.columns.tolist(),  
    class_names=['Class 0', 'Class 1'],  
    mode='classification'  
)

str1 = ["Max_1HR", "Max_1HR_2", "Max_1HR_72", "Max_1TD", 'RH_Count', 'TD_Count', 'EV', 'TWI', 'DTW']

feature_importance_df = pd.DataFrame(columns=str1)

for i in range(len(X_test)):
    
    exp = explainer.explain_instance(
        X_test.iloc[i].values, 
        model_rf_new.predict_proba, 
        num_features=len(str1)
    )
    
   
    feature_importance_scores = {feature: 0 for feature in str1}
    for feature, importance in exp.as_list():
        for feature_name in str1:
            if feature_name in feature: 
                feature_importance_scores[feature_name] += importance  

    
    feature_importance_df.loc[len(feature_importance_df)] = feature_importance_scores

# save the result
feature_importance_df.to_csv('feature_important_lime_rf_240328.csv', index=False)

print(feature_importance_df.head())